In [12]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [21]:
w3 = Web3(HTTPProvider("https://1rpc.io/eth"))

In [22]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984", # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 10000000000000000000
slippage = int(0.20 * 10000) # Convert to bps

In [23]:
with open("ethereum_wallet.json", "r") as f:
    keypair_json = json.load(f)

private_key = Account.decrypt(keypair_json, "")
eth_wallet = Account.from_key(private_key)

In [36]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/permit2/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,          # wallet is your LocalAccount
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [37]:
print ("Gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

Gas fee: 0.000507313970804443 ETH


In [38]:
# 1.  Sign **exactly** the 32‑byte digest in that quote
digest   = to_bytes(hexstr=quote["permit2"]["hash"])         # 32 bytes

key_obj  = keys.PrivateKey(eth_wallet.key)                       # raw priv‑key
raw_sig  = key_obj.sign_msg_hash(digest)                     # r,s,v   (v = 0/1)
signature = raw_sig.to_bytes() 

assert len(signature) == 65

In [39]:
# 2.  Prove locally that the router will recover the right address

msg       = SignableMessage(b"\x00", b"", digest)            # raw‑hash envelope
recovered = Account.recover_message(msg, signature=signature)

if recovered.lower() != eth_wallet.address.lower():
    display ("❌ signer mismatch!")

'❌ signer mismatch!'

In [40]:
# 3.  Append length + signature to the calldata
calldata = (
    bytes.fromhex(quote["transaction"]["data"][2:]) +        # original data
    len(signature).to_bytes(32, "big") +                     # 0x…41
    signature
)

In [41]:
# 4. Build, sign, send
tx = {
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     "0x" + calldata.hex(),
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "gasPrice": int(quote["transaction"]["gasPrice"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address),
    "chainId":  1,
}

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

In [42]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex) 
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0xb7a4f0f77defb05f6458b8e3194a1f064ba5ccef808ef467403fc4716d5078a0
⏳ waiting …
✅ status   : 0


In [45]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0xb7a4f0f77defb05f6458b8e3194a1f064ba5ccef808ef467403fc4716d5078a0
